In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import utils as utl
#from collections import Counter



ImportError: Traceback (most recent call last):
  File "/home/dklee/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/dklee/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/dklee/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/dklee/anaconda3/envs/tensorflow/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/dklee/anaconda3/envs/tensorflow/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcublas.so.10.0: cannot open shared obje

In [ ]:
train_x, val_x, test_x, train_y, val_y, test_y = utl.train_val_test_split(seq_data, labels, split_frac=0.80)
print("Data Set Size")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

In [ ]:
    def model_inputs():
        inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
        labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
        keep_prob_ = tf.placeholder(tf.float32, name='keep_prob')

        return inputs_, labels_, keep_prop_


    def build_lstm_layers(lstm_sizes, inputs_, keep_prob_, batch_size):
        """
        Create the LSTM Layers
        """
        lstms = [tf.contrib.rnn.BasicLSTMCell(size) for size in lstm_sizes]

        drops = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob_) for lstm in lstms]
        cell = tf.contrib.rnn.MultiRNNCell(drops)
        initial_state = cell.zero_state(batch_size, tf.float32)
        lstm_outputs, final_state = tf.nn.dynamic_rnn(cell, inputs_, initial_state=initial_state)

    def build_cost_fn_and_opt(lstm_outputs, labels_, learning_rate):
        """
        Create the Loss Function and Optimizer
        """

        predictions = tf.contrib.layers.fully_connected(lstm_outputs[:, -1], 1, activation_fn=tf.sigmoid)
        loss = tf.losses.mean_squared_error(labels_, predictions)
        optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)

    def build_accuracy(predictions, labels_):

        """
        Create accuracy
        """

        correct_prd = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    def build_and_train_network(lstm_sizes, vocab_size, epochs, batch_size, 
                                learning_rate, keep_prob, train_x, val_x, train_y, val_y):
        inputs_, labels_, keep_prob_ = model_inputs()
        initial_state, lstm_outputs, lstm_cell, final_state = build_lstm_layers(lstm_sizes, inputs_, keep_prob_, batch_size)
        predictions, loss, optimizer = build_cost_fn_and_opt(lstm_outputs, labels_, learning_rate)
        accuracy = build_accuracy(predictions, labels_)

        saver = tf.train.Saver()

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            n_batches = len(train_x)//batch_size
            for e in range(epochs):
                state = sess.run(initial_state)

                train_acc = []
                for ii, (x, y) in enumerate(utl.get_batches(train_x, train_y, batch_size), 1):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                           keep_prob_: keep_prob,
                           initial_state: state}
                    loss_, state, _, batch_acc = sess.run([loss, final_state, optimizer, accuracy], feed_dict=feed)
                    train_acc.append(batch_acc)

                    if (ii + 1) % n_batches == 0:

                        val_acc = []
                        val_state = sess.run(lstm_cell.zero_state(batch_size, tf.float32))
                        for xx, yy in utl.get_batches(val_x, val_y, batch_size):
                            feed = {inputs_: xx,
                                    labels_: yy[: None],
                                   keep_prob_: 1,
                                   initial_state: val_state}
                            val_batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                            val_acc.append(val_batch_acc)

                        print("Epoch: {}/{}...".format(e+1, epochs),
                              "Batch: {}/{}...".format(ii+1, n_batches),
                              "Train Loss: {:.3f}...".format(loss_),
                              "Train Accruacy: {:.3f}...".format(np.mean(train_acc)),
                              "Val Accuracy: {:.3f}".format(np.mean(val_acc)))
            saver.save(sess, "checkpoints/thydys.ckpt")


In [ ]:
# Define Inputs and Hyperparameters
lstm_sizes = [128, 64]
epochs = 50
batch_size = 24
learning_rate = 0.1
keep_prob = 0.5
with tf.Graph().as_default():
    build_and_train_network(lstm_sizes, epochs, batch_size,
                            learning_rate, keep_prob, train_x, val_x, train_y, val_y)